In [ ]:
import matplotlib.pyplot as plt
import re
import numpy as np
import pandas as pd
import seaborn as sns
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('uncore-frequency-switching-latency'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
SOURCE_FREQUENCY = "Source Frequency [MHz]"
TARGET_FREQUENCY = "Target Frequency [MHz]"

In [ ]:
#returns for log a dict of dict of dicts:
# resulting_dict[source-frequency][target-frequency][metric]
# metric can be "t_gap": the gap time during the switch of frequencies
#               "t_init": how long it took to initialize the frequency switch
#               "source_p": the performance of the workload before the assumed frequency switch
#               "target_p": the performance of the workload after the assumed frequency switch
# the latter metrics can be used to check for sanity
# returns the pair of (all data, data with 1%, 50% and 99% quantile)
def read_log_file(filename):

    first_phase_regex='^(\d+) MHz->(\d+)Mhz Cycles per access before:(\d+) after:(\d+), switch after (\d+) cycles, took (\d+) cycles, switch timestamp before (\d+), after (\d+)$'

    prog_1=re.compile(first_phase_regex)

    f=open(filename)
    data = []
    for line in f:
        result=re.match(prog_1, line)
        if not result:
            continue

        source=int(result.group(1))
        target=int(result.group(2))
        source_p=int(result.group(3))
        target_p=int(result.group(4))
        t_init=float(result.group(5))/2E3
        t_gap=float(result.group(6))/2E3
        time_before=int(result.group(7))
        time_after=int(result.group(8))

        data.append({
            SOURCE_FREQUENCY: source,
            TARGET_FREQUENCY: target,
            't_init': t_init,
            't_gap': t_gap,
            'source_p': source_p,
            'target_p': target_p,
            'time_before': time_before,
            'time_after': time_after
        })

    f.close()

    data_all = pd.DataFrame(data)

    data_all['wait_latency_before'] = data_all["time_before"].diff() / 2E3
    data_all['wait_latency_after'] = data_all["time_after"].diff() / 2E3

    data_all['nb_of_accesses_before_gap'] = data_all['t_init'] * 2E3 / data_all['source_p']

    # Filter out invalid values where the source_p or target_p is zero
    data_all = data_all[data_all['source_p'] != 0]
    data_all = data_all[data_all['target_p'] != 0]

    # discard the first row
    data_all = data_all.iloc[1:]

    data_quantile = pd.DataFrame()

    for source in data_all[SOURCE_FREQUENCY].unique():
        for target in data_all[TARGET_FREQUENCY].unique():
            data = data_all[data_all[SOURCE_FREQUENCY] == source]
            data = data[data[TARGET_FREQUENCY] == target]
            data_quantile = pd.concat([data_quantile, pd.DataFrame([
                {'median_t_init': np.median(data['t_init'].quantile(0.5)),
                '1percentQuantile_t_init': data['t_init'].quantile(0.01),
                '95percentQuantile_t_init': data['t_init'].quantile(0.95),
                '99percentQuantile_t_init': data['t_init'].quantile(0.99),
                'median_t_gap': np.median(data['t_gap'].quantile(0.5)),
                '1percentQuantile_t_gap': data['t_gap'].quantile(0.01),
                '95percentQuantile_t_gap': data['t_gap'].quantile(0.95),
                '99percentQuantile_t_gap': data['t_gap'].quantile(0.99),
                SOURCE_FREQUENCY: source,
                TARGET_FREQUENCY: target}
            ])], ignore_index=True)

    return data_all, data_quantile

In [ ]:
def plot_heatmap(data,metric,metric_long,vmin,vmax,plt_filename):
    plt.rcParams['figure.figsize'] = 5,5

    ax = sns.heatmap(data=data.pivot(index=SOURCE_FREQUENCY, columns=TARGET_FREQUENCY, values=metric),
                     vmin=vmin, vmax=vmax, square=True, cmap="viridis", linecolor="black", linewidths=0.5,
                     cbar_kws={'label': metric_long})
    ax.invert_yaxis()

    Plotting.savefig(experiment, plt_filename, annotations_y_offset=0.025, annotations_y_spacing=0.025)

In [ ]:
data_all, data_quantile = read_log_file(f'{experiment.path}/manual.2000.out')
data_all

In [ ]:
data_quantile

In [ ]:
# The set of all performance values
# The detection threshold has to be higher than all performance values in cycles
all_performances = set(data_all['source_p'].tolist()).union(set(data_all['target_p']))
print(f"Lowest performance in cycles is {max(all_performances)}")
print(f"Highest performance in cycles is {min(all_performances)}")

In [ ]:
plot_heatmap(data_quantile, "1percentQuantile_t_init", "1% percentile of $t_{init}$ [µs]", 0, 1000, "1percentile-t-init.pdf")

In [ ]:
plot_heatmap(data_quantile, "median_t_init", "Median of $t_{init}$ [µs]", 0, 1000, "median-t-init.pdf")

In [ ]:
plot_heatmap(data_quantile, "95percentQuantile_t_init", "95% quantile of $t_{init}$ [µs]", 0, 1000, "95percentile-t-init.pdf")

In [ ]:
plot_heatmap(data_quantile, "95percentQuantile_t_gap", "95% quantile of $t_{gap}$ [µs]", 0, 10, "95percentile-t-gap.pdf")

In [ ]:
for source_freq in data_all[SOURCE_FREQUENCY].unique():
    filtered_data = data_all
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, xlim=(0, 250), ylim=(0, 250), col_wrap=4)
    g.map(sns.scatterplot, 'source_p', 'target_p', alpha=0.3)
    g.add_legend()

In [ ]:
for source_freq in data_all[SOURCE_FREQUENCY].unique():
    filtered_data = data_all
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, xlim=(0, 1000), ylim=(0, 10), col_wrap=4)
    g.map(sns.scatterplot, 't_init', 't_gap', alpha=0.3)
    g.add_legend()

In [ ]:
for source_freq in data_all[SOURCE_FREQUENCY].unique():
    filtered_data = data_all
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, xlim=(0, 10), col_wrap=4)
    g.map(sns.ecdfplot, 't_gap')
    g.add_legend()
    for axi in g.axes:
        axi.axvline(x=4, color='black', dashes=[2], linewidth=1)

In [ ]:
for source_freq in data_all[SOURCE_FREQUENCY].unique():
    filtered_data = data_all
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, xlim=(0, 1000), col_wrap=4)
    g.map(sns.ecdfplot, 't_init')
    g.add_legend()
    for axi in g.axes:
        axi.axvline(x=575, color='black', dashes=[2], linewidth=1)


In [ ]:
for source_freq in data_all[SOURCE_FREQUENCY].unique():
    filtered_data = data_all
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, xlim=(0, 10000), height=5, col_wrap=4)
    g.map(sns.scatterplot, 'wait_latency_before', 't_init', alpha=0.3)
    g.add_legend()

In [ ]:
for source_freq in data_all[SOURCE_FREQUENCY].unique():
    filtered_data = data_all
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, xlim=(0, 10000), height=5, col_wrap=4)
    g.map(sns.scatterplot, 'wait_latency_after', 't_init', alpha=0.3)
    g.add_legend()

In [ ]:
for source_freq in data_all[SOURCE_FREQUENCY].unique():
    filtered_data = data_all
    filtered_data = filtered_data[filtered_data[SOURCE_FREQUENCY] == source_freq]

    g = sns.FacetGrid(filtered_data, col=TARGET_FREQUENCY, height=5, col_wrap=4)
    g.map(sns.ecdfplot, 'nb_of_accesses_before_gap')
    g.add_legend()